# Reinforcement Learning Homework

Reinforcement learning (RL) is interested in how we can design algorithms that get better at making decisions in dynamic environments. In this homework, we'll focus on **policy gradient** algorithms. Reinforcement learning contains a plethora of algorithm components and types, but the policy gradient family is easier to explain and get some intuition on.

## Policy Gradient

The idea behind policy gradient is similar to how you would approach learning any new skill like riding a bike, baking a cake, etc. You try out some actions based on what you think is most likely to work, you reflect a little bit on what you're lacking or could improve on, and you try again. In short, trial and error. To make this work as an algorithm, we need to start specifying some components.

Recall that the **policy** is the agent's decision making strategy, often modeled as a probability distribution which we label $\pi$. We might write the joint distribution $\pi(s, a)$ or a conditional distribution $\pi(a \mid s)$. In either case, the policy is the answer the question, "If you, the agent, were in scenario $s$, how likely are you to take action $a$?" 

If we model the policy as a neural network, then there's a direct connection between the weights of the neural network and how well the policy performs. Our goal is to then determine a way by which we can tweak the weights to improve performance.

In supervised learning, we have the idea of **gradient descent** as a way to improve a model's performance on a classification or regression task by modifying the weights to lower some measure of model error. We can apply the exact same idea and strategy but it requires us to create a measure of performance.

## Proximal Policy Optimization (PPO)

In this homework, we'll be looking at an instantiation of policy gradient called [Proximal Policy Optimization](https://openai.com/blog/openai-baselines-ppo/). The paper is [here](https://arxiv.org/pdf/1707.06347.pdf) -- part of your assignment is to read or at least fully skim it.


The idea in this paper is to introduce a slightly different objective; figuring out the reasoning behind this change is part of your assignment, but the paper explains why they make this change. The objective that they are deviating from looks like

$$
L^{CPI}(\theta) = \hat{\mathbb{E}}_t\left[\frac{\pi_\theta(a_t \mid s_t) }{ \pi_{\theta_{old}}(a_t \mid s_t) } \hat{A}_t \right]
$$

The $\hat{\mathbb{E}}_t$ represents an empirical expectation (i.e. an average that we will compute) over time. $\theta$ represents weights of the neural network so that $\theta_{old}$ are the weights we would like to improve upon. $\hat{A}_t$ represents the **advantage** over time. 

The advantage $A$ is a numerical score assigned to state-action pairs $(s, a)$. If you want a little mathematical notation to go with that,

$$
A: \mathcal{S}\times\mathcal{A} \to \mathbb{R}.
$$

The advantage's job is to say how good it is to be in a certain state taking a certain action. We won't go into how it's computed, but just know that if somebody gave you a record of all the states they encountered, actions they took, and rewards they achieved, that would be enough to compute the advantage. The purpose of introducing an advantage function is to make it easy to identify which state-action pairs we should focus on making more likely.

# Assignment

This assignment will be fairly gentle since next week's homework is also about RL. The idea is just to get you accustomed to thinking about the objects and data structures we need to do RL.

## Task 1

Please read [this brief documentation](https://gym.openai.com/docs/) about OpenAI Gym, which provides lots of environments to test RL algorithms. Here are two questions to figure out during/after reading:

1. Assume you have an action that you want to take (for example, you might call `sample` from the `env.action_space` to get a uniformly random action). How do you actually take this action in the environment? What method do you call on the `env`?
2. How do you reset an environment? What information does the reset return?

## Task 2

First, we will import some libraries. We will need to install OpenAI `spinningup`, but this has to be done by cloning their repo. You might want to delete this cell after the first time you run it.

In [ ]:
!git clone https://github.com/openai/spinningup
!cd spinningup
!pip install -e spinningup

fatal: destination path 'spinningup' already exists and is not an empty directory.
Obtaining file:///content/spinningup
  Found existing installation: spinup 0.2.0
    Can't uninstall 'spinup'. No files were found to uninstall.
  Running setup.py develop for spinup


You may need to restart the Colab kernel (Runtime > restart runtime) for spinup to become part of the libraries.

In [ ]:
import numpy as np
import torch
from torch.optim import Adam
import gym
import time
import spinup.algos.pytorch.ppo.core as core
from spinup.utils.logx import EpochLogger
from spinup.utils.mpi_pytorch import setup_pytorch_for_mpi, sync_params, mpi_avg_grads
from spinup.utils.mpi_tools import mpi_fork, mpi_avg, proc_id, mpi_statistics_scalar, num_procs

Most RL algorithms use a buffer data structure to record the information from trial episodes. Take a look at the methods to get a sense for what this buffer does.

In [ ]:
class PPOBuffer:
    def __init__(self, obs_dim, act_dim, size, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros(core.combined_shape(size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros(core.combined_shape(size, act_dim), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.
        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """

        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)
        
        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = core.discount_cumsum(deltas, self.gamma * self.lam)
        
        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = core.discount_cumsum(rews, self.gamma)[:-1]
        
        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick
        adv_mean, adv_std = mpi_statistics_scalar(self.adv_buf)
        self.adv_buf = (self.adv_buf - adv_mean) / adv_std
        data = dict(obs=self.obs_buf, act=self.act_buf, ret=self.ret_buf,
                    adv=self.adv_buf, logp=self.logp_buf)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in data.items()}

## Task 3

Okay, here's the part where you will start implementing things. There are two losses that you need to have.

1. A loss for the policy. We are implementing policy gradient after all.
2. A loss for the **value function**, which is related to the advantage function. 

The difference is that value is a function of state only whereas advantage is a function of both state and action. Again, you don't need to worry about the details of how value is computed. 

First, we'll walk through implementing the policy loss. This is Equation 7 in the paper. A few important notes to consider:

* The way this code is structured, we use the [log of probability](https://en.wikipedia.org/wiki/Log_probability), which can be more stable than probabilities, which have to be in the $[0,1]$ interval. So if you want to compute $r_t = \frac{\pi(a \mid s)}{ \pi_{\text{old}}(a \mid s) }$ but you only have access to $\log \pi (a \mid s)$ and $\log \pi_{\text{old}}(a \mid s)$, what do you do? In the code block below, put this value into the `ratio` variable using only 1 call to `torch.exp`.

In [ ]:
def your_policy_loss(ac, obs, act, adv, logp_old, epsilon):
    pi, logp = ac.pi(obs, act)
    ratio    = torch.exp(logp-logp_old) # TODO compute the ratio using log probs and 1 call to torch.exp
    clip_adv = torch.clamp(ratio, 1-epsilon, 1+epsilon) * adv # TODO appropriately clip the ratio and multiply it by the advantage. Use torch.clamp to clip.
    loss_pi  = -torch.min(clip_adv, ratio*adv).mean() # TODO find the mean of the minimum of clip_adv and ratio * adv. Hint: use torch.min and then call .mean() on the result
    
    # NOTE make sure to put a negative sign in front of the loss because pytorch tries to maximize, not minimize, by default.

    return pi, ratio, loss_pi, logp

The value function is represented by a neural network too, but we have the supervision for it! Write a mean squared error loss for the value assigned to a state/observation if the correct value is `ret`. 

In [ ]:
def your_value_loss(ac, obs, ret):
    value_fn = ac.v
    value = value_fn(obs) # TODO compute the value of obs
    return ((value-ret)**2).mean() # TODO square the difference between value and ret and call .mean() on that value.

In [ ]:
def ppo(env_fn, actor_critic=core.MLPActorCritic, ac_kwargs=dict(), seed=0, 
        steps_per_epoch=4000, epochs=50, gamma=0.99, clip_ratio=0.2, pi_lr=3e-4,
        vf_lr=1e-3, train_pi_iters=80, train_v_iters=80, lam=0.97, max_ep_len=1000,
        target_kl=0.01, logger_kwargs=dict(), save_freq=10):
    """
    Proximal Policy Optimization (by clipping), 
    with early stopping based on approximate KL
    Args:
        env_fn : A function which creates a copy of the environment.
            The environment must satisfy the OpenAI Gym API.
        actor_critic: The constructor method for a PyTorch Module with a 
            ``step`` method, an ``act`` method, a ``pi`` module, and a ``v`` 
            module. The ``step`` method should accept a batch of observations 
            and return:
            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``a``        (batch, act_dim)  | Numpy array of actions for each 
                                           | observation.
            ``v``        (batch,)          | Numpy array of value estimates
                                           | for the provided observations.
            ``logp_a``   (batch,)          | Numpy array of log probs for the
                                           | actions in ``a``.
            ===========  ================  ======================================
            The ``act`` method behaves the same as ``step`` but only returns ``a``.
            The ``pi`` module's forward call should accept a batch of 
            observations and optionally a batch of actions, and return:
            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``pi``       N/A               | Torch Distribution object, containing
                                           | a batch of distributions describing
                                           | the policy for the provided observations.
            ``logp_a``   (batch,)          | Optional (only returned if batch of
                                           | actions is given). Tensor containing 
                                           | the log probability, according to 
                                           | the policy, of the provided actions.
                                           | If actions not given, will contain
                                           | ``None``.
            ===========  ================  ======================================
            The ``v`` module's forward call should accept a batch of observations
            and return:
            ===========  ================  ======================================
            Symbol       Shape             Description
            ===========  ================  ======================================
            ``v``        (batch,)          | Tensor containing the value estimates
                                           | for the provided observations. (Critical: 
                                           | make sure to flatten this!)
            ===========  ================  ======================================
        ac_kwargs (dict): Any kwargs appropriate for the ActorCritic object 
            you provided to PPO.
        seed (int): Seed for random number generators.
        steps_per_epoch (int): Number of steps of interaction (state-action pairs) 
            for the agent and the environment in each epoch.
        epochs (int): Number of epochs of interaction (equivalent to
            number of policy updates) to perform.
        gamma (float): Discount factor. (Always between 0 and 1.)
        clip_ratio (float): Hyperparameter for clipping in the policy objective.
            Roughly: how far can the new policy go from the old policy while 
            still profiting (improving the objective function)? The new policy 
            can still go farther than the clip_ratio says, but it doesn't help
            on the objective anymore. (Usually small, 0.1 to 0.3.) Typically
            denoted by :math:`\epsilon`. 
        pi_lr (float): Learning rate for policy optimizer.
        vf_lr (float): Learning rate for value function optimizer.
        train_pi_iters (int): Maximum number of gradient descent steps to take 
            on policy loss per epoch. (Early stopping may cause optimizer
            to take fewer than this.)
        train_v_iters (int): Number of gradient descent steps to take on 
            value function per epoch.
        lam (float): Lambda for GAE-Lambda. (Always between 0 and 1,
            close to 1.)
        max_ep_len (int): Maximum length of trajectory / episode / rollout.
        target_kl (float): Roughly what KL divergence we think is appropriate
            between new and old policies after an update. This will get used 
            for early stopping. (Usually small, 0.01 or 0.05.)
        logger_kwargs (dict): Keyword args for EpochLogger.
        save_freq (int): How often (in terms of gap between epochs) to save
            the current policy and value function.
    """

    # Special function to avoid certain slowdowns from PyTorch + MPI combo.
    setup_pytorch_for_mpi()

    # Set up logger and save configuration
    logger = EpochLogger(**logger_kwargs)
    logger.save_config(locals())

    # Random seed
    seed += 10000 * proc_id()
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Instantiate environment
    env = env_fn()
    obs_dim = env.observation_space.shape
    act_dim = env.action_space.shape

    # Create actor-critic module
    ac = actor_critic(env.observation_space, env.action_space, **ac_kwargs)

    # Sync params across processes
    sync_params(ac)

    # Count variables
    var_counts = tuple(core.count_vars(module) for module in [ac.pi, ac.v])
    logger.log('\nNumber of parameters: \t pi: %d, \t v: %d\n'%var_counts)

    # Set up experience buffer
    local_steps_per_epoch = int(steps_per_epoch / num_procs())
    buf = PPOBuffer(obs_dim, act_dim, local_steps_per_epoch, gamma, lam)
    
    def compute_loss_pi(data):
        obs, act, adv, logp_old = data['obs'], data['act'], data['adv'], data['logp']

        # Policy loss
        pi, ratio, loss_pi, logp = your_policy_loss(ac, obs, act, adv, logp_old, clip_ratio)

        # Useful extra info
        approx_kl = (logp_old - logp).mean().item()
        ent = pi.entropy().mean().item()
        clipped = ratio.gt(1+clip_ratio) | ratio.lt(1-clip_ratio)
        clipfrac = torch.as_tensor(clipped, dtype=torch.float32).mean().item()
        pi_info = dict(kl=approx_kl, ent=ent, cf=clipfrac)

        return loss_pi, pi_info

    def compute_loss_v(data):
        obs, ret = data['obs'], data['ret']
        return your_value_loss(ac, obs, ret)

    # Set up optimizers for policy and value function
    pi_optimizer = Adam(ac.pi.parameters(), lr=pi_lr)
    vf_optimizer = Adam(ac.v.parameters(), lr=vf_lr)

    # Set up model saving
    logger.setup_pytorch_saver(ac)
    
    def mystery():
        data = buf.get()

        pi_l_old, pi_info_old = compute_loss_pi(data)
        pi_l_old = pi_l_old.item()
        v_l_old = compute_loss_v(data).item()

        for i in range(train_pi_iters):
            pi_optimizer.zero_grad()
            loss_pi, pi_info = compute_loss_pi(data)
            kl = mpi_avg(pi_info['kl'])
            if kl > 1.5 * target_kl:
                logger.log('Early stopping at step %d due to reaching max kl.'%i)
                break
            loss_pi.backward()
            mpi_avg_grads(ac.pi)
            pi_optimizer.step()

        logger.store(StopIter=i)

        for i in range(train_v_iters):
            vf_optimizer.zero_grad()
            loss_v = compute_loss_v(data)
            loss_v.backward()
            mpi_avg_grads(ac.v)
            vf_optimizer.step()

        kl, ent, cf = pi_info['kl'], pi_info_old['ent'], pi_info['cf']
        logger.store(LossPi=pi_l_old, LossV=v_l_old,
                     KL=kl, Entropy=ent, ClipFrac=cf,
                     DeltaLossPi=(loss_pi.item() - pi_l_old),
                     DeltaLossV=(loss_v.item() - v_l_old))
    
    # Prepare for interaction with environment
    start_time = time.time()
    o = env.reset()
    ep_ret = 0
    ep_len = 0

    # Main loop: collect experience in env and update/log each epoch
    for epoch in range(epochs):
        for t in range(local_steps_per_epoch):
            a, v, logp = ac.step(torch.as_tensor(o, dtype=torch.float32))
            
            # TODO how do we take an action in the environment
            # ===============================
            f = env.step
            # ===============================
            
            next_o, r, d, _ = f(a)
            ep_ret += r
            ep_len += 1

            # save and log
            buf.store(o, a, r, v, logp)
            logger.store(VVals=v)
            
            # Update obs (critical!)
            o = next_o

            timeout = ep_len == max_ep_len
            terminal = d or timeout
            epoch_ended = t==local_steps_per_epoch-1

            if terminal or epoch_ended:
                if epoch_ended and not(terminal):
                    print('Warning: trajectory cut off by epoch at %d steps.'%ep_len, flush=True)
                # if trajectory didn't reach terminal state, bootstrap value target
                if timeout or epoch_ended:
                    _, v, _ = ac.step(torch.as_tensor(o, dtype=torch.float32))
                else:
                    v = 0
                buf.finish_path(v)
                if terminal:
                    # only save EpRet / EpLen if trajectory finished
                    logger.store(EpRet=ep_ret, EpLen=ep_len)

                o = env.reset()
                ep_ret = 0
                ep_len = 0


        # Save model
        if (epoch % save_freq == 0) or (epoch == epochs-1):
            logger.save_state({'env': env}, None)

        mystery()

        # Log info about epoch
        logger.log_tabular('Epoch', epoch)
        logger.log_tabular('EpRet', with_min_and_max=True)
        logger.log_tabular('EpLen', average_only=True)
        logger.log_tabular('VVals', with_min_and_max=True)
        logger.log_tabular('TotalEnvInteracts', (epoch+1)*steps_per_epoch)
        logger.log_tabular('LossPi', average_only=True)
        logger.log_tabular('LossV', average_only=True)
        logger.log_tabular('DeltaLossPi', average_only=True)
        logger.log_tabular('DeltaLossV', average_only=True)
        logger.log_tabular('Entropy', average_only=True)
        logger.log_tabular('KL', average_only=True)
        logger.log_tabular('ClipFrac', average_only=True)
        logger.log_tabular('StopIter', average_only=True)
        logger.log_tabular('Time', time.time()-start_time)
        logger.dump_tabular()

Run your algorithm! You're not expected to show that you can solve the environment, but the goal of this assignment is to see a cool example of RL so run this for as long as is feasible for you. I picked two simple environments to play around with: Cart Pole and Lunar Lander. Lunar Lander needs about 60 epochs before you see reasonable behavior, CartPole only needs 20 or so. Feel free to look at Gym for more environments, your only task is to comment on the behavior you see for whatever environment you pick.

In [ ]:
env_fn = lambda : gym.make('CartPole-v1')
env_fn = lambda : gym.make('LunarLander-v2')

ac_kwargs = dict(
        hidden_sizes=[64,64],
        activation=torch.nn.ReLU,
        )

logger_kwargs = dict(output_dir='./output/', exp_name='my-first-rl-experiment')

ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=5000, epochs=60, logger_kwargs=logger_kwargs)

Logging data to ./output/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "activation":	"ReLU",
        "hidden_sizes":	[
            64,
            64
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "clip_ratio":	0.2,
    "env_fn":	"<function <lambda> at 0x7f46a85831e0>",
    "epochs":	60,
    "exp_name":	"my-first-rl-experiment",
    "gamma":	0.99,
    "lam":	0.97,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x7f46a2b2e630>":	{
            "epoch_dict":	{},
            "exp_name":	"my-first-rl-experiment",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"./output/",
            "output_file":	{
                "<_io.TextIOWrapper name='./output/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"
                }
            }
        }
    },
    "logger_kwargs":	{
        "exp_name":	"my-first-rl-experiment",
        "output_dir":	"./output/

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


---------------------------------------
|             Epoch |               0 |
|      AverageEpRet |            -201 |
|          StdEpRet |             114 |
|          MaxEpRet |           -75.8 |
|          MinEpRet |            -496 |
|             EpLen |            93.2 |
|      AverageVVals |          -0.154 |
|          StdVVals |          0.0497 |
|          MaxVVals |           -0.07 |
|          MinVVals |          -0.447 |
| TotalEnvInteracts |           5e+03 |
|            LossPi |        6.36e-08 |
|             LossV |        1.51e+04 |
|       DeltaLossPi |         -0.0147 |
|        DeltaLossV |       -2.85e+03 |
|           Entropy |            1.38 |
|                KL |          0.0115 |
|          ClipFrac |           0.126 |
|          StopIter |              79 |
|              Time |            4.97 |
---------------------------------------
---------------------------------------
|             Epoch |               1 |
|      AverageEpRet |            -135 |


Unfortunately, it doesn't seem like the spinup render works easily with Jupyter notebooks or Colab notebooks, but we do want to see the results of your work! Please see if you can do the following:

1. download the output folder to your computer
2. Install spinup the same way we did earlier in this notebook on your command line
3. Run the command below on your command line

If not, you can use the `progress.txt` file in the output folder to demonstrate the performance of your algorithm.

In [ ]:
!python -m spinup.run test_policy ./output/



Loading from ./output/pyt_save/model.pt.


/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
Logging data to /tmp/experiments/1605331567/progress.txt
Traceback (most recent call last):
  File "/content/spinningup/spinup/utils/test_policy.py", line 153, in <module>
    run_policy(env, get_action, args.len, args.episodes, not(args.norender))
  File "/content/spinningup/spinup/utils/test_policy.py", line 121, in run_policy
    env.render()
  File "/usr/local/lib/python3.6/dist-packages/gym/core.py", line 235, in render
    return self.env.render(mode, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/gym/envs/box2d/lunar_lander.py", line 320, in render
    from gym.envs.classic_control import rendering
  File "/usr/local/lib/python3.6/dist-packages/gym/envs/classic_control/rendering.py", line 27, in <module>
    from pyglet.gl import *
  F